## icmecat

makes the ICMECATv2.0

Author: C. Moestl, IWF Graz, Austria
twitter @chrisoutofspace, part of https://github.com/cmoestl/heliocats
last update March 2020

Install a specific conda environment to run this code, see https://github.com/cmoestl/heliocats

**current status: work in progress**

* to do: despike sta stb wind all, new STA, Wind and PSP converted to SCEQ components 

* Convert to script with 

    jupyter nbconvert --to script icmecat.ipynb
    
    
* Adding a new event: edit the file icmecat/HELCATS_ICMECAT_v20_master.xlsx to add 3 times, the id and spacecraft name, 
delete the file for the respective spacecraft under data/indices_icmecat, and run this notebook or script.


In [1]:
import numpy as np
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import  DateFormatter
import seaborn as sns
import datetime
import astropy.constants as const
from sunpy.time import parse_time
import time
import pickle
import sys
import os
import urllib
import json
import importlib
import pandas as pd
import copy
import openpyxl


from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging

from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import cats as hc
importlib.reload(hc) #reload again while debugging

#where the 6 in situ data files are located is read from input.py
#as data_path=....
from input import data_path


########### make directories first time if not there

resdir='results'
if os.path.isdir(resdir) == False: os.mkdir(resdir)

datadir='data'
if os.path.isdir(datadir) == False: os.mkdir(datadir)

indexdir='data/indices_icmecat' 
if os.path.isdir(indexdir) == False: os.mkdir(indexdir) 


catdir='icmecat'
if os.path.isdir(catdir) == False: os.mkdir(catdir)

icplotsdir='icmecat/plots_icmecat/' 
if os.path.isdir(icplotsdir) == False: os.mkdir(icplotsdir) 

### (1) load new data with HelioSat and heliocats.data

In [2]:
   
load_data=1

if load_data > 0:

    print('load new Wind, STEREO-A, MAVEN, Parker Solar Probe data')

    # MAVEN
    #filemav='maven_2014_2018.p'
    #[mav,hmav]=pickle.load(open(filemav, 'rb' ) )

    #filemav='maven_2014_2018_removed.p'
    #[mav,hmav]=pickle.load(open(filemav, 'rb' ) )
    
    filemav='maven_2014_2018_removed_smoothed.p'
    [mav,hmav]=pickle.load(open(data_path+filemav, 'rb' ) )

    # Wind
    filewin="wind_2018_2019.p" 
    #for updating data
    #start=datetime.datetime(2018, 1, 1)
    #end=datetime.datetime.utcnow()
    #hd.save_wind_data(data_path,filewin,start,end)
    [win2,hwin2]=pickle.load(open(data_path+filewin, "rb" ) )  

    # STEREO-A    
    filesta2='sta_2018_2019_beacon.p'
    #start=datetime.datetime(2018, 1, 1)
    #end=datetime.datetime(2019, 12, 31)
    #hd.save_stereoa_beacon_data(data_path,filesta,start,end)
   
    [sta2,hsta2]=pickle.load(open(data_path+filesta2, "rb" ) )  

    # Parker Solar Probe
    filepsp='psp_2018_2019.p'
    [psp,hpsp]=pickle.load(open(data_path+filepsp, "rb" ) )  


    # ADD BepiColombo  
    
    
    # ADD Solar Orbiter
    
    
    # Ulysses is currently taken from the full 
    # helcats data below, but a file is available on figshare


    # get data file from helcats with headers
    [vex,win,mes,sta,stb,uly,hvex,hwin,hmes,hsta,hstb,huly]=hd.load_helcats_datacat(data_path+'helcats_all_data_removed.p') 
    print('done')

load new Wind, STEREO-A, MAVEN, Parker Solar Probe data
load all helcats DATACAT from single file:  /nas/helio/data/insitu_python/helcats_all_data_removed.p
Use vex,win,sta,stb,mes,uly to access data and position, hvex,hwin, hmes, hsta, hstb, huly for headers.
done


## (2) measure new events 

In [3]:
#for measuring new events use this function from heliocats.plot 
#plt.close('all')
#works in jupyter notebooks

#works in scripts
#matplotlib.use('qt5agg')  
#plt.ion()
#hp.plot_insitu_measure(psp, '2018-Nov-10','2018-Nov-15', 'PSP', 'results/plots_icmecat/')
#%matplotlib 
#hp.plot_insitu_measure(win, '2010-Oct-29','2010-Oct-31', 'Wind', 'results/')


#for plotting single events
#hp.plot_insitu(psp, ic.icme,'2018-Nov-15', 'PSP', icplotsdir)

### (3) make ICMECAT 

In [5]:
print('data loaded')
ic=hc.load_helcats_icmecat_master_from_excel('icmecat/HELCATS_ICMECAT_v20_master.xlsx')

####### 3a get indices for all spacecraft

wini=np.where(ic.sc_insitu == 'Wind')[:][0] 
stai=np.where(ic.sc_insitu == 'STEREO-A')[:][0]    
stbi=np.where(ic.sc_insitu == 'STEREO-B')[:][0]    
vexi=np.where(ic.sc_insitu == 'VEX')[:][0]  
mesi=np.where(ic.sc_insitu == 'MESSENGER')[:][0]   
ulyi=np.where(ic.sc_insitu == 'ULYSSES')[:][0]    
mavi=np.where(ic.sc_insitu == 'MAVEN')[:][0]    
pspi=np.where(ic.sc_insitu == 'PSP')[:][0]    

####### 3b get parameters for all spacecraft one after another

ic=hc.get_cat_parameters(win,wini,ic,'Wind')
ic=hc.get_cat_parameters(sta,stai,ic,'STEREO-A')
ic=hc.get_cat_parameters(stb,stbi,ic,'STEREO-B')
ic=hc.get_cat_parameters(vex,vexi,ic,'VEX')
ic=hc.get_cat_parameters(mes,mesi,ic,'MESSENGER')
ic=hc.get_cat_parameters(uly,ulyi,ic,'ULYSSES')
ic=hc.get_cat_parameters(mav,mavi,ic,'MAVEN')
ic=hc.get_cat_parameters(psp,pspi,ic,'PSP')


####### 3c make all plots if wanted
# matplotlib.use('Agg')
# hp.plot_icmecat_events(win,wini,ic,'Wind',icplotsdir)
# hp.plot_icmecat_events(sta,stai,ic,'STEREO-A',icplotsdir)
# hp.plot_icmecat_events(stb,stbi,ic,'STEREO-B',icplotsdir)
# hp.plot_icmecat_events(vex,vexi,ic,'VEX',icplotsdir)
# hp.plot_icmecat_events(mes,mesi,ic,'MESSENGER',icplotsdir)
# hp.plot_icmecat_events(uly,ulyi,ic,'ULYSSES',icplotsdir)
# hp.plot_icmecat_events(mav,mavi,ic,'MAVEN',icplotsdir)
# hp.plot_icmecat_events(psp,pspi,ic,'PSP',icplotsdir)


data loaded
load HELCATS ICMECAT from file: icmecat/HELCATS_ICMECAT_v20_master.xlsx
Get parameters for  Wind
Get parameters for  STEREO-A
Get parameters for  STEREO-B
Get parameters for  VEX
Get parameters for  MESSENGER
Get parameters for  ULYSSES
Get parameters for  MAVEN
Get parameters for  PSP


### (4) save ICMECAT 

In [67]:
#save header and parameters as text file and prepare for html website

header='ICME CATALOGUE v2.0 \n\n\
This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetic field and bulk plasma observations in the heliosphere. \n\n\
This is version 2.0, released in April 2020 with a major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). \n\n\
Released 2020-**-**. DOI: 10.6084/m9.figshare.4588315.v2 *** \n\
The catalog is available as .p (python pickle), .xlsx, .csv, .html, .json at https://helioforecast.space/catalogs \n\n\
Number of events in ICMECAT: '+str(len(ic))+' \n\
ICME observatories: Wind, STEREO-A, STEREO-B, Venus Express (VEX), MESSENGER, Ulysses, MAVEN, Parker Solar Probe (PSP).   \n\
Time range: January 2007 - December 2018. \n \n\
Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria.\n\
Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott, \
Vratislav Krupar, Jonathan Eastwood, Simon Good, Lan Jian, Teresa Nieves-Chinchilla, Jingnan Guo, Mateja Dumbovic, Benoit Lavraud.  \n\n\
This catalog has been made by getting the 3 times of each ICME (shock or disturbance begin, \
magnetic obstacle start and end) from the individual catalogs below, and then calculating all parameters again consistently from the data by us. \n\
Each icmecat_id has a tag in it that indicates from which catalog the ICME times were taken. \n\n\
Wind:       Nieves-Chinchilla et al. (2018), tag: NASA. \n\
STEREO-A:   Jian et al. (2018), tag: JIAN. \n\
STEREO-B:   Jian et al. (2018), tag: JIAN. \n\
VEX:        Good et al. (2018), tag: SGOOD \n\
MESSENGER:  Good et al. (2018), Winslow et al. (2018), tags: SGOOD, WINSLOW. \n\
MAVEN:      Möstl et al. (2020, in prep.), tag: MOESTL.\n\
Ulysses:    Added by us, tag: MOESTL. \n\
PSP:        Added by us, tag: MOESTL. \n\
We have also added extra events at Ulysses, STEREO-A, PSP, MESSENGER, VEX, Wind and MAVEN (all tagged with MOESTL in icmecat_id).\n\n\
Comments: \n\
- Spacecraft positions are given in Heliocentric Earth Equatorial Coordinates (HEEQ) coordinates. \n\
- Coordinate system for all magnetic field components is SCEQ except Ulysses and Parker Solar Probe (RTN), MAVEN (MSO) and Wind (HEEQ). \n\
        Definition of SpaceCraft Equatorial Coordinates (SCEQ): \n\
        Z is the solar rotation axis. \n\
        X points from the Sun to the spacecraft, projected in the solar equatorial plane. \n\
        Y completes the right handed triad and points to solar west. \n\
        This system is thus centered on the respective in situ spacecraft. \n\
        The solar equatorial plane as the reference plane is similar for all spacecraft.\n\
- Venus Express and MESSENGER do not have plasma parameters available. \n\
- The identification of ICMEs for MAVEN is a mixture of methods using data from MSL/RAD, MAVEN and STEREO HI (Möstl et al. 2020, in prep.). \n\n\
The in situ data (4 GB) can be downloaded in python pickle format as recarrays from https://doi.org/10.6084/m9.figshare.11973693 \n\
The python code for producing this catalog is available as part of https://github.com/cmoestl \n\n\
References: \n\
Nieves-Chinchilla, T. et al. (2018), https://doi.org/10.1007/s11207-018-1247-z  https://wind.nasa.gov/fullcatalogue.php \n\
Jian, L. et al. (2018), https://doi.org/10.3847/1538-4357/aab189 https://stereo-ssc.nascom.nasa.gov/data/ins_data/impact/level3/ \n\
Good, S. et al. (2018) https://doi.org/10.1007/s11207-015-0828-3 \n\
Winslow, R. et al. (2015), https://doi.org/10.1002/2015JA021200 \n\
Möstl, C. et al. (2020) in preparation \n\n\n\n'


     
# Coordinate system for all spacecraft except Ulysses (RTN) and MAVEN (MSO):  SCEQ
           
# SpaceCraft Equatorial Coordinates (SCEQ):
# 	Z is the solar rotation axis.
# 	X points from the Sun to the spacecraft, projected in the solar equatorial plane. 
# 	Y completes the right handed triad and points to solar west.
#  This system is thus centered on the respective in situ spacecraft. 
#  The solar equatorial plane as the reference plane is similar for all spacecraft.
           
# Notes: 
# (1) For all spacecraft: If there is no sheath region, so the ICME starts immediately with a magnetic obstacle, the ICME_START_TIME is similar to MO_START_TIME.
# (2) For all spacecraft the times that are given are: the ICME_START_TIME (defined by shock or elevated magnetic field start), the MO_START_TIME,
#     and the MO_END_TIME (MO for magnetic obstacle, defined by elevated field and smooth magnetic field rotations, but also includes (few) complex ejecta).
#     Only for Wind the ICME_END_TIME is given extra, which contains interval of disturbed solar wind by the ICME until the values return to slow solar wind.
# (3) MESSENGER: For ICME events catalogued by Winslow et al. the end time of the ICME is used for MO_END_TIME (not ICME_END_TIME). 
#     Caution: after 2011 March 18 when MESSENGER is in orbit around Mercury the times are sometimes not exact because 
# (4) MESSENGER and VEX: For events cataloged by Simon Good ICME_START_TIME has been added by V. Krupar (Imperial College) and C. Moestl (Graz).
# (5) For the calculation of the parameters at MESSENGER during the orbit around Mercury, all data points inside the bowshock of Mercury have been removed.
#     (according to a list thankfully provided to us by by R. Winslow, UNH, B. Anderson, APL, Lydia Philpott, UBC).
# (6) For MVA to be applied to the MO interval, up to 25% of the MO data points may be NaN. Otherwise MVA results are set to NaN.
#     This is relevant to MESSENGER in orbit around Mercury and VEX at Venus, where the ICME observation in the solar wind sometimes contains too few data points.
# (7) Calculation of the MO parameters at VEX is done with an approximate removal of the induced magnetosphere, with a modified equation
#     according to the one in the discussion of Zhang et al. 2008 (doi: 10.1016/j.pss.2007.09.012), with a constant of 3.5 instead of 2.14/2.364,
#     in order to account for a larger bowshock distance during solar maximum than studied in this paper. 
# (8) Similarly, for MAVEN data, all data inside the bow shock were removed with the model from Edberg et al. 2008 (JGR), including one standard deviation.
#     For the remaining data the median for each orbit is taken as 1 data point. 4.5 hour time resolution.



parameters='Parameters:\n\n\
01: icmecat_id: The unique identifier for the observed ICME. unit: string. \n\
02: icme_start_time: Shock arrival or density enhancement time, can be similar to mo_start_time. unit: UTC. \n\
03: mo_start_time: Start time of the magnetic obstacle (MO), including flux ropes, flux-rope-like, and ejecta signatures. unit: UTC. \n\
04: mo_end_time: End time of the magnetic obstacle. unit: UTC. \n\
05: mo_sc_heliodistance: Heliocentric distance of the spacecraft at mo_start_time. unit: AU.\n\
06: mo_sc_long_heeq: Heliospheric longitude of the spacecraft at mo_start_time, range [-180,180]. unit: degree (HEEQ).\n\
07: mo_sc_lat_heeq: Heliospheric latitude of the spacecraft at mo_start_time, range [-90,90]. unit: degree (HEEQ).\n\
08: icme_duration: Duration of the interval between icme_start_time and mo_endtime. unit: hours.\n\
09: icme_bmax: Maximum total magnetic field in the full icme interval (icme_start_time to mo_end_time). unit: nT.\n\
10: icme_bmean: Mean total magnetic field during the full icme interval (icme_start_time to mo_end_time). unit: nT.\n\
11: icme_bstd: Standard deviation of the total magnetic field from icme_start_time to mo_end_time. unit: nT.\n\
12: icme_speed_mean: Mean proton speed from icme_start_time to mo_end_time. unit: km/s.\n\
13: icme_speed_std: Standard deviation of proton speed from icme_start_time to mo_end_time. unit: km/s.\n\
14: mo_duration: Duration of the interval between mo_start_time and mo_endtime. unit: hours.\n\
15: mo_bmax: Maximum total magnetic field in the magnetic obstacle interval (mo_start_time to mo_end_time). unit: nT.\n\
16: mo_bmean: Mean total magnetic field in the magnetic obstacle. unit: nT.\n\
17: mo_bstd: Standard deviation of the total magnetic field in the magnetic obstacle. unit: nT.\n\
18: mo_bzmean: Mean magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
19: mo_bzmin: Minimum magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
20: mo_bzstd: Standard deviation of the magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
21: mo_bymean: Mean magnetic field By component in the magnetic obstacle. unit: nT.\n\
22: mo_bystd: Standard deviation of the magnetic field By component in the magnetic obstacle. unit: nT.\n\
23: mo_speed_mean: Mean proton speed from mo_start_time to mo_end_time. unit: km/s.\n\
24: mo_speed_std: Standard deviation of proton speed from mo_start_time to mo_end_time. unit: km/s.\n\
25: mo_expansion_speed: Difference between proton speed at mo_start_time to proton speed at mo_end_time. unit: km/s.\n\
26: mo_pdyn_mean: Mean proton dynamic pressure from mo_start_time to mo_start_time. unit: nPa.\n\
27: mo_pdyn_std: Standard deviation of proton dynamic pressure from mo_start_time to mo_start_time. unit: nPa.\n\
28: mo_density_mean: Mean proton density from mo_start_time to mo_start_time. unit: cm^-3.\n\
29: mo_density_std: Standard deviation of proton density from mo_start_time to mo_start_time. unit: cm^-3.\n\
30: mo_temperature_mean: Mean proton temperature from mo_start_time to mo_start_time. unit: K.\n\
31: mo_temperature_std: Standard deviation of proton temperature from mo_start_time to mo_end_time. unit: K.\n\
32: sheath_speed_mean: Mean proton speed from icme_start_time to mo_start_time, NaN if these times are similar. unit: km/s.\n\
33: sheath_speed_std: Standard deviation of proton speed from icme_start_time to mo_start_time, NaN if these times are similar. unit: km/s.\n\
34: sheath_density_mean: Mean proton density from icme_start_time to mo_start_time, NaN if these times are similar. unit: cm^-3.\n\
35: sheath_density_std: Standard deviation of proton density from icme_start_time to mo_start_time, NaN if these times are similar. unit: cm^-3.\n\
36: sheath_pdyn_mean: Mean proton dynamic pressure, from icme_start_time to mo_start_time, NaN if these times are similar. unit: nPa.\n\
37: sheath_pdyn_std: Standard deviation of proton dynamic pressure, from icme_start_time to mo_start_time, NaN if these times are similar. unit: nPa.'


# header_html='<p> ICME CATALOGUE v2.0 <br /> <br /> \
# This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetometer and plasma observations in the heliosphere.<br /> <br /> \
# Version 2.0, released in March 2020 with major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). <br /><br />  \
# Number of events in ICMECAT: '+str(len(ic))+' <br /> <br /> \
# ICME observatories: Wind, Parker Solar Probe, STEREO-A, STEREO-B, Venus Express, MESSENGER, MAVEN, ULYSSES <br /> \
# Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria,<br /> \
# Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott,<br /> \
# Vratislav Krupar, Jonathan Eastwood, Simon Good, Lan Jian, Teresa Nieves-Chinchilla. <br /> <br /> \
# Time range: January 2007 - December 2018. <br /> <br /> \
# This is version: 2.0 of the catalogue, released 2020-??-??. DOI: 10.6084/m9.figshare.4588315.v2 <p>'

header_html=''

parameters_html=''



print(header)
print(parameters)

#make header file
file='icmecat/HELCATS_ICMECAT_v20_header.txt'
with open(file, "w") as text_file:
    text_file.write(header)
    text_file.write(parameters)

    
print()    
print()    
print()    

print('header saved as '+file)

ICME CATALOGUE v2.0 

This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetic field and bulk plasma observations in the heliosphere. 

This is version 2.0, released in April 2020 with a major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). 

Released 2020-**-**. DOI: 10.6084/m9.figshare.4588315.v2 *** 
The catalog is available as .p (python pickle), .xlsx, .csv, .html, .json at https://helioforecast.space/catalogs 

Number of events in ICMECAT: 678 
ICME observatories: Wind, STEREO-A, STEREO-B, Venus Express (VEX), MESSENGER, Ulysses, MAVEN, Parker Solar Probe (PSP).   
Time range: January 2007 - December 2018. 
 
Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria.
Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott, Vratislav Krupar, Jonathan Eastwood, Simon Good, Lan Jian, Teresa Nieves-

In [6]:
### save ICMECAT as pickle with times as datetime objects
file='icmecat/HELCATS_ICMECAT_v20.p'
pickle.dump(ic, open(file, 'wb'))
print('ICMECAT saved as '+file)



################ save to different formats

#copy pandas dataframe first to change time format

ic_copy=copy.deepcopy(ic)  
ic_copy.icme_start_time=parse_time(ic.icme_start_time).isot
ic_copy.mo_start_time=parse_time(ic.mo_start_time).isot
ic_copy.mo_end_time=parse_time(ic.mo_end_time).isot

#change time format
for i in np.arange(len(ic)):

    dum=ic_copy.icme_start_time[i] 
    ic_copy.at[i,'icme_start_time']=dum[0:16]
     
    dum=ic_copy.mo_start_time[i] 
    ic_copy.at[i,'mo_start_time']=dum[0:16]
     
    dum=ic_copy.mo_end_time[i] 
    ic_copy.at[i,'mo_end_time']=dum[0:16]


#save as Excel
file='icmecat/HELCATS_ICMECAT_v20.xlsx'
ic_copy.to_excel(file,sheet_name='ICMECATv2.0')
print('ICMECAT saved as '+file)

#save as json
file='icmecat/HELCATS_ICMECAT_v20.json'
ic_copy.to_json(file)
print('ICMECAT saved as '+file)

#save as csv
file='icmecat/HELCATS_ICMECAT_v20.csv'
ic_copy.to_csv(file)
print('ICMECAT saved as '+file)

#save as html
file='icmecat/HELCATS_ICMECAT_v20_simple.html'
ic_copy.to_html(file)
print('ICMECAT saved as '+file)

#save as hdf needs pip install tables
#file='icmecat/HELCATS_ICMECAT_v20.hdf'
#ic.to_hdf(file,key='icmecat')

#save as .mat does not work yet
#ile='icmecat/HELCATS_ICMECAT_v20.mat'
#icdict=ic.to_dict()
#scipy.io.savemat(file,ic.values)

#save as txt
file='icmecat/HELCATS_ICMECAT_v20.txt'
np.savetxt(file, ic_copy.values.astype(str), fmt='%s' )

print('ICMECAT saved as '+file)

ic2=hc.load_icmecat()



############ save as html file for helioforecast.space

file='icmecat/HELCATS_ICMECAT_v20.p'
ic=pickle.load( open(file, 'rb'))

#save as html
file='icmecat/HELCATS_ICMECAT_v20.html'
#ic.to_html(file,justify='center')

ichtml='{% extends "_base.html" %} \n \n {% block content %} \n \n \n '
ichtml  += header_html
ichtml += ic.to_html()
ichtml +='\n \n {% endblock %}'

'''
{% extends "_base.html" %}
{% block content %}
<div class="row">
    <div class="col-12 col-md-12 col-xl-12">
        <p> </br>
        Recent real-time solar and solar wind data from SDO in Earth orbit, 
        from the DSCOVR or ACE spacecraft at the Sun-Earth L1 point provided by NOAA and from STEREO-A.
        Updated twice daily at 05:00 and 17:00 UTC.</br></br>
        </p>
    </div>
</div>
'''



with open(file,'w') as f:
    f.write(ichtml)
    f.close()
    
print('ICMECAT saved as '+file)    

ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.p
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.xlsx
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.json
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.csv
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20_simple.html
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.txt
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.html
